# 03A: DLT Autoloader - Raw Sensor Data
This notebook ingests raw sensor CSV data into a Delta table using Auto Loader wrapped in DLT.


In [0]:
import dlt
from pyspark.sql.types import *

# Define schema for raw sensor data
sensor_schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("aircraft_id", StringType(), True),
    StructField("model", StringType(), True),
    StructField("engine_temp", DoubleType(), True),
    StructField("fuel_efficiency", DoubleType(), True),
    StructField("vibration", DoubleType(), True),
    StructField("altitude", DoubleType(), True),
    StructField("airspeed", DoubleType(), True),
    StructField("anomaly_score", DoubleType(), True)
])

volume_path = "/Volumes/arao/aerodemo/tmp/raw"

@dlt.table(
    name="raw_sensor_data",
    comment="Raw sensor readings ingested via Auto Loader"
)
def load_raw_sensor_data():
    return (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("header", "true")
            .option("pathGlobFilter", "*.csv")
            .option("cloudFiles.schemaLocation", f"{volume_path}/schema/raw_sensor_data")
            .schema(sensor_schema)
            .load(volume_path)
    )
